In [1]:
# !pip install pandas
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.optim as optim

In [2]:
dt1=pd.read_excel('Book1.xlsx')
print(dt1.head())

         ph       Solids  Turbidity  Potability
0  7.080795  20791.31898   2.963135           1
1  3.716080  18630.05786   4.500656           0
2  8.099124  19909.54173   3.055934           1
3  8.316766  22018.41744   4.628771           1
4  9.092223  17978.98634   4.075075           0


In [3]:
dt1.columns

Index(['ph', 'Solids', 'Turbidity', 'Potability'], dtype='object')

In [4]:
dt1=dt1[['ph','Solids','Turbidity','Potability']]
dt1.head(10)

,ph,Solids,Turbidity,Potability
0,7.080795,20791.31898,2.963135,1
1,3.716080,18630.05786,4.500656,0
2,8.099124,19909.54173,3.055934,1
3,8.316766,22018.41744,4.628771,1
4,9.092223,17978.98634,4.075075,0
5,5.584087,28748.68774,2.559708,0
6,10.223862,28749.71654,2.672989,0
7,8.635849,13672.09176,4.401425,0
8,7.080795,14285.58385,3.595017,1
9,11.180284,25484.50849,4.370562,0


In [5]:
dt1.describe()

,ph,Solids,Turbidity,Potability
count,3276.000000,3276.000000,3276.000000,3276.000000
mean,7.080795,22014.092526,3.966786,0.667888
std,1.469956,8768.570828,0.780382,0.471043
min,0.000000,320.942611,1.450000,0.000000
25%,6.277673,15666.690300,3.439711,0.000000
50%,7.080795,20927.833605,3.955028,1.000000
75%,7.870050,27332.762125,4.500320,1.000000
max,14.000000,61227.196010,6.739000,1.000000


In [6]:
dt1.isnull().sum()

ph            0
Solids        0
Turbidity     0
Potability    0
dtype: int64

In [8]:
dt1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ph          3276 non-null   float64
 1   Solids      3276 non-null   float64
 2   Turbidity   3276 non-null   float64
 3   Potability  3276 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 102.5 KB


In [9]:
dt1.isnull().sum()

ph            0
Solids        0
Turbidity     0
Potability    0
dtype: int64

In [10]:
dt1.describe()

,ph,Solids,Turbidity,Potability
count,3276.000000,3276.000000,3276.000000,3276.000000
mean,7.080795,22014.092526,3.966786,0.667888
std,1.469956,8768.570828,0.780382,0.471043
min,0.000000,320.942611,1.450000,0.000000
25%,6.277673,15666.690300,3.439711,0.000000
50%,7.080795,20927.833605,3.955028,1.000000
75%,7.870050,27332.762125,4.500320,1.000000
max,14.000000,61227.196010,6.739000,1.000000


In [11]:
X = dt1.drop('Potability', axis=1).values
y = dt1['Potability'].values

In [12]:
X

array([[7.08079500e+00, 2.07913190e+04, 2.96313538e+00],
       [3.71608007e+00, 1.86300579e+04, 4.50065627e+00],
       [8.09912419e+00, 1.99095417e+04, 3.05593375e+00],
       ...,
       [9.41951032e+00, 3.31555782e+04, 3.29887550e+00],
       [5.12676292e+00, 1.19838694e+04, 4.70865847e+00],
       [7.87467136e+00, 1.74041771e+04, 2.30914906e+00]])

In [13]:
y

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [14]:
from collections import Counter

d = dict(Counter(y))
d

{1: 2188, 0: 1088}

In [15]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[ 2.86744064e-07, -1.39470871e-01, -1.28629758e+00],
       [-2.28933943e+00, -3.85986649e-01,  6.84217891e-01],
       [ 6.92867739e-01, -2.40047337e-01, -1.16736546e+00],
       ...,
       [ 1.59125363e+00,  1.27080989e+00, -8.56006782e-01],
       [-1.32951598e+00, -1.14405809e+00,  9.50797384e-01],
       [ 5.40150854e-01, -5.25811938e-01, -2.12445866e+00]])

In [16]:
df = pd.DataFrame(X)

In [17]:
df.describe()

,0,1,2
count,3.276000e+03,3.276000e+03,3.276000e+03
mean,-4.965503e-16,3.185622e-18,-3.521806e-16
std,1.000153e+00,1.000153e+00,1.000153e+00
min,-4.817747e+00,-2.474344e+00,-3.225560e+00
25%,-5.464412e-01,-7.239916e-01,-6.755095e-01
50%,2.867441e-07,-1.238999e-01,-1.507005e-02
75%,5.370063e-01,6.066532e-01,6.837866e-01
max,4.707802e+00,4.472689e+00,3.552921e+00


In [18]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

In [19]:
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [20]:
for i in range(len(train_dataset)):
    inputs, labels = train_dataset[i]
    # print("Inputs:", inputs)
    # print("Labels:", labels)

In [21]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:

class WaterPotabilityNN(nn.Module):
    def __init__(self):
        super(WaterPotabilityNN, self).__init__()
        self.fc1 = nn.Linear(3, 128)
#         self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 256)
#         self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 1024)
        self.fca1 = nn.Linear(1024, 2048)
        self.fca2 = nn.Linear(2048, 1024)
        self.fc5 = nn.Linear(1024, 512)
#         self.dropout3 = nn.Dropout(0.2)
        self.fc6 = nn.Linear(512, 256)
        
        # self.dropout4 = nn.Dropout(0.1)
        self.fc7 = nn.Linear(256, 128)
        self.fc8 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
#         x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
#         x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
#         x = self.dropout3(x)
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fca1(x))
        x = torch.relu(self.fca2(x))
        # x = self.dropout4(x)
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.relu(self.fc7(x))        
        x = torch.sigmoid(self.fc8(x))
        return x


model = WaterPotabilityNN()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

criterion = nn.MSELoss()  # Binary Cross-Entropy Loss
optimizer = optim.RMSprop(model.parameters(), lr=0.000001)



5576961


In [23]:
def evaluate(model, data_loader, criterion):
    model.eval()
    losses = []
    num_samples = 0

    for i, (images, classes) in enumerate(data_loader):
        num_samples += images.shape[0]
        c = model(images)
        loss = criterion(c, classes)
        losses.append(loss.item() * images.shape[0])

    loss = sum(losses) / num_samples
    model.train()
    return loss

In [24]:
num_epochs = 200
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        
        loss.backward()
        optimizer.step()
    
    epoch_loss = evaluate(model, train_loader, criterion)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}')



# Evaluating the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/200, Loss: 0.24688322029950965
Epoch 2/200, Loss: 0.24646142345803385
Epoch 3/200, Loss: 0.24607130072954048
Epoch 4/200, Loss: 0.2456686356822953
Epoch 5/200, Loss: 0.24524875503914956
Epoch 6/200, Loss: 0.2448206131012385
Epoch 7/200, Loss: 0.2443617571401232
Epoch 8/200, Loss: 0.24388901880224242
Epoch 9/200, Loss: 0.24339151414296098
Epoch 10/200, Loss: 0.24288711336277824
Epoch 11/200, Loss: 0.24233370896514136
Epoch 12/200, Loss: 0.2417343751847289
Epoch 13/200, Loss: 0.24107610069158422
Epoch 14/200, Loss: 0.24036085953694264
Epoch 15/200, Loss: 0.2395590019590072
Epoch 16/200, Loss: 0.23864567407214915
Epoch 17/200, Loss: 0.23759902904506858
Epoch 18/200, Loss: 0.2363622108033595
Epoch 19/200, Loss: 0.23505127323037794
Epoch 20/200, Loss: 0.2335742200603922
Epoch 21/200, Loss: 0.2319899512383774
Epoch 22/200, Loss: 0.23026202835199486
Epoch 23/200, Loss: 0.22837757224800023
Epoch 24/200, Loss: 0.22644038885149337
Epoch 25/200, Loss: 0.22445042499604115
Epoch 26/200, Los

In [25]:
# Evaluating the model
model.eval()
accuracies = 0
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        # print(outputs)
        # print(labels)
        predicted = (outputs > 0.5).float()
        # print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = correct / total
        accuracies+=accuracy    
    accuracies = accuracies/len(test_loader)
    print(f'Test Accuracy: {accuracies * 100:.2f}%')

Test Accuracy: 81.17%


In [26]:
scripted_model = torch.jit.script(model)
scripted_model.save("water_potability.pt")